# Testing HuggingFace DistilBERT model

## 1. Import packages

In [ ]:
import wandb
import numpy as np
import argparse
from datasets import Value, Sequence
from nesta_ds_utils.loading_saving import S3
from discovery_child_development.getters.binary_classifier.embeddings_hugging_face import (
    get_embeddings,
)
from discovery_child_development.getters.binary_classifier.prompts_edge_cases import get_examples
from discovery_child_development.utils.huggingface_pipeline import (
    load_model,
    load_training_args,
    load_trainer,
    saving_huggingface_model
)
from discovery_child_development.utils import wandb as wb
from discovery_child_development.utils import classification_utils
from discovery_child_development.utils.testing_examples_utils import testing_examples_huggingface
from discovery_child_development import (
    logging,
    S3_BUCKET,
    config,
    binary_config,
    PROJECT_DIR
)

## 2. Setting parameters

In [ ]:
# Set up
S3_PATH = "models/binary_classifier/"
SEED = config["seed"]
NUM_SAMPLES = config["embedding_sample_size"]
# Set the seed
np.random.seed(SEED)

In [ ]:
#PARAMS
identifier = "50"
wandb_run = False
save_model = False
production = False

## 3. Load data

In [ ]:
# Loading the training and validation embeddings
embeddings_training = get_embeddings(
    identifier=identifier, production=production, set_type="train"
)
embeddings_validation = get_embeddings(
    identifier=identifier, production=production, set_type="validation"
)

In [ ]:
examples = get_examples()

## 4. Training the model

In [ ]:
if wandb_run:
    print("Logging in wandb")
    run = wandb.init(
        project="ISS supervised ML",
        job_type="Binary classifier - huggingface",
        save_code=True,
        tags=["huggingface", "binary_classifier", "sentence_embeddings"],
    )

In [ ]:
# Load the model
model = load_model(config=binary_config, num_labels=2)

# Train model with early stopping
training_args = load_training_args(output_dir=S3_PATH, config=binary_config)
trainer = load_trainer(
    model=model,
    args=training_args,
    train_dataset=embeddings_training,
    eval_dataset=embeddings_validation,
    config=binary_config,
)
trainer.train()

## 5. Evaluating the model

In [ ]:
# Evaluate model
trainer.evaluate()

# View f1, prediction, recall and accuracy of predictions on validation set
model_predictions = trainer.predict(embeddings_validation)

In [ ]:
model_predictions.metrics

In [ ]:
predictions = np.argmax(model_predictions.predictions, axis=-1)
labels = model_predictions.label_ids.ravel().tolist()
# Creating confusion matrix
confusion_matrix = classification_utils.plot_confusion_matrix(
    labels, predictions, None, "Relevant works"
)

In [ ]:
# Saving the model and logging to wandb
if save_model:
    # Save model to S3
    SAVE_TRAINING_RESULTS_PATH = PROJECT_DIR / "outputs/data/models/"
    saving_huggingface_model(trainer, 
                             f"binary_classifier_distilbert_{identifier}_production_{production}", 
                             save_path=SAVE_TRAINING_RESULTS_PATH, 
                             s3_path=S3_PATH)

In [ ]:
if wandb_run:
    # Log metrics
    wandb.run.summary["f1"] = model_predictions.metrics["test_f1"]
    wandb.run.summary["accuracy"] = model_predictions.metrics["test_accuracy"]
    wandb.run.summary["precision"] = model_predictions.metrics["test_precision"]
    wandb.run.summary["recall"] = model_predictions.metrics["test_recall"]

    # Adding reference to this model in wandb
    wb.add_ref_to_data(
        run=run,
        name=f"binary_classifier_{model}_" + identifier,
        description=f"{model} model trained on binary classifier training data",
        bucket=S3_BUCKET,
        filepath=f"{S3_PATH}binary_classifier_{model}_{identifier}.pkl",
    )

    # Log confusion matrix
    wb_confusion_matrix = wandb.Table(
        data=confusion_matrix, columns=confusion_matrix.columns
    )
    run.log({"confusion_matrix": wb_confusion_matrix})

    # End the weights and biases run
    wandb.finish()

## 6. Trialing some examples

In [ ]:
# Removing Not-specified
examples = examples.query("labels!='Not specified'")
examples.tail()

In [ ]:
testing_examples_huggingface(trainer,examples, binary_config)